# Pre Processing

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx to Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx
User uploaded file "Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx" with length 6481955 bytes


In [12]:
import pandas as pd
import numpy as np

#Read dataset
features =  pd.read_excel("Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx")

Replace code features with the name, to use get_dummies() function and obtain a one-hot-encoding

In [13]:
features["Diploma_scuola_superiore"] = features["Diploma_scuola_superiore"].map({1: "Classico", 2: "Scientifico", 3: "Linguistico", 4: "Magistrale", 5: "Artistico", 6: "Tecnico", 7: "Professionale", 8: "Altro_italiano", 9: "Estero", 99: "Non_disponibile"})
features["area_geografica_scuolasuperiore"] = features["area_geografica_scuolasuperiore"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["area_geografica_residenza"] = features["area_geografica_residenza"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["Ambito"] = features["Ambito"].map({1: "Economia", 2: "Farmacia", 3: "Giurisprudenza", 4: "Ingegneria", 5: "Lingue", 6: "Medicina", 7: "Veterinaria", 8: "Psicologia", 9: "Scienze", 10: "Scienze_agroalimanetari", 11: "Scienze_educazione_formazione", 12: "Scienze_motorie", 13: "Scienze_politiche", 14: "Scienze_statistiche", 15: "Sociologia", 16: "Studi_umanistici"})
features = pd.get_dummies(features)

Abbiamo sostituito i valori NaN della feature "voto_scuola_superiore" con la media. Inoltre sono state droppate le colonne non utili ai fini dell'addestramento, ed infine eliminate le tuple contente almeno un NaN

In [14]:
from datetime import datetime, date

def getAge(dataNascita, coorte):
    ts = pd.to_datetime(str(dataNascita)) 
    born = ts.strftime('%d/%m/%Y')
    born = datetime.strptime(born, "%d/%m/%Y").date()
    date = "31/12/"+str(coorte)
    dataAttuale = datetime.strptime(date, "%d/%m/%Y").date()
    age = dataAttuale.year - born.year - ((dataAttuale.month, dataAttuale.day) < (born.month, born.day))
    return age

#add new column with NaN
features["Eta"] = np.nan

for ind in features.index:
    studente = features.loc[ind]
    dataNascita = studente["DataNascita"]
    coorte = studente["Coorte"]
    #fill the new column with age 
    features.at[ind,'Eta'] = getAge(dataNascita, coorte)


In [15]:
mean_value = features['voto_scuola_superiore'].mean()
features['voto_scuola_superiore'] = features['voto_scuola_superiore'].fillna(mean_value)
features= features.drop('ID_Studente', axis = 1)
features= features.drop('Classe_ISEE', axis = 1)
features= features.drop('DataNascita', axis = 1)
features=features.drop('OFA_superati',axis=1)
features = features.drop('Coorte', axis = 1)
features = features.dropna()

Abbiamo bilanciato il dataset con approccio downsampling pareggiando il numero di dropout e non droput

## Bilanciamento 1

In [16]:
#Data preprocessing
dropout = pd.DataFrame()
noDropout = pd.DataFrame()

for ind in features.index:
    studente = pd.DataFrame(features.loc[[ind]])
    if features["Abbandoni"][ind] == 1:
        dropout = pd.concat([studente, dropout], ignore_index=True, axis = 0)
    else:
        noDropout = pd.concat([studente, noDropout], ignore_index=True, axis = 0)

#Bilanciamento dataset
final = pd.DataFrame()
for ind in dropout.index:
    studente_drop = pd.DataFrame(dropout.loc[[ind]])
    studente_noDrop = pd.DataFrame(noDropout.loc[[ind]])
    final = pd.concat([studente_drop, final], ignore_index=True, axis = 0)
    final = pd.concat([studente_noDrop, final], ignore_index=True, axis = 0)

#Dividing the data into attributes and labels
X = final.drop('Abbandoni', axis=1)
y = final['Abbandoni']
#Dividing data into training and test sets


##Bilanciamento 2

In [6]:
#Dividing the data into attributes and labels
X = features.drop('Abbandoni', axis=1)
y = features['Abbandoni']

#BILANCIAMENTO 2
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

#Resampling data
rus = RandomUnderSampler(sampling_strategy = "majority", random_state=42)
X_res, y_res = rus.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

X = X_res
y = y_res

Resampled dataset shape Counter({0: 2093, 1: 2093})


# Algorithm

Split the training testing set with a 10 % size and print the dimension of the data splitted
Split arrays or matrices into random train and test subsets.

In [17]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import metrics


X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((3348, 59), (838, 59), (3348,), (838,))

Build a model and fit it with training test

In [18]:
model = linear_model.LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

Predict a value in Y_pred using model defined previously

In [19]:
Y_pred = model.predict(X_test)

#Evaluating

In [20]:
print('Coefficients: ',model.coef_)
print('Intercept: ',model.intercept_)
print('Mean Squared Error :%.2f' % mean_squared_error(Y_test,Y_pred))
print('Coefficient of determination (R ^2): %.2f' % r2_score(Y_test,Y_pred))

Coefficients:  [[ 4.95114643e-04 -1.39327259e-02 -8.49112576e-06  6.79917909e-03
   2.64440679e-05 -3.38274248e-04 -5.60631022e-04 -3.51222230e-04
   1.38274181e-04  7.18093520e-04 -9.24928822e-04  2.88488196e-04
  -6.92719222e-04  8.94466937e-04 -2.61971316e-03  1.70143627e-03
  -1.69095598e-03  3.86299492e-03  6.74821058e-04  2.55702984e-03
   3.14273782e-04 -2.61934498e-03  6.83440178e-04  6.52169591e-05
   6.97925763e-05  3.45887798e-03  1.80365554e-04  0.00000000e+00
   2.93743759e-04 -2.32734304e-03  8.07180416e-03 -6.83432562e-03
   4.37958295e-04 -1.24806751e-03  8.01369193e-05  1.87804275e-03
  -1.14498785e-04 -8.29841346e-04 -5.69617432e-04 -5.11022836e-04
   2.98708324e-03 -1.19652471e-04 -6.37949250e-04 -2.44719658e-04
  -1.69908221e-03  2.41058532e-04 -3.93833432e-04  2.84469889e-03
   1.27014323e-05  3.37911131e-03 -6.16883622e-04  1.63375828e-04
  -8.50226505e-05 -2.24045287e-03  2.53336264e-04 -1.50674396e-04
   7.14817486e-04  2.57829483e-04  5.15343915e-02]]
Intercept

Evaluating the algorithm

we compute the confusion matrix, but it can't operate with continuos value, we must convert in [0,1].we say that any value in Y_pred above (0.6,1) convert to 1 otherwise 0

In [21]:
print("Evaluating the Algorithm:\n")
cf_matrix = metrics.confusion_matrix(Y_test,Y_pred)
print(cf_matrix)
print(metrics.classification_report(Y_test,Y_pred))


print("true negative",cf_matrix[0,0])
print("false negative",cf_matrix[1,0])
print("true positive",cf_matrix[1,1])
print("false positive",cf_matrix[0,1])

print("\nEvaluating Prediction Accuracy:\n")
print("Accuracy:", metrics.accuracy_score(Y_test, Y_pred))

Evaluating the Algorithm:

[[261 176]
 [179 222]]
              precision    recall  f1-score   support

           0       0.59      0.60      0.60       437
           1       0.56      0.55      0.56       401

    accuracy                           0.58       838
   macro avg       0.58      0.58      0.58       838
weighted avg       0.58      0.58      0.58       838

true negative 261
false negative 179
true positive 222
false positive 176

Evaluating Prediction Accuracy:

Accuracy: 0.5763723150357996


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Confusion Matrix AVGExams');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Real Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()